### Play with networktables to see how the API is organized  - 20220313 CJH

In [38]:
import time
import networktables
import pandas as pd
from _pyntcore._ntcore import NetworkTableType

In [39]:
ntinst = networktables.NetworkTablesInstance.getDefault()

#### can give nt multiple servers to try

In [40]:
servers = ["127.0.0.1", "10.24.29.2"] #, "roboRIO-2429-FRC.local"]  # need to add the USB one here
ntinst.startClient(servers=servers)

#### if it connects you can get the connection info - may have to wait a few seconds though

In [41]:
connections = ntinst.getConnections()

In [42]:
print(f'ID:{connections[0].remote_id}\tIP:{connections[0].remote_ip}')

ID:Robot	IP:127.0.0.1


#### looks like all entries are given their own subtable

In [43]:
table = ntinst.getGlobalTable()  # also NetworkTables.getTable('/')
table

In [45]:
test_obj = table
attribs = [attrib for attrib in dir(test_obj) if '__' not in attrib]
_ = [print(attrib, end=', ') if (ix+1)%8!=0 else print(attrib, end='\n') for ix, attrib in enumerate(attribs) ]

PATH_SEPARATOR_CHAR, addEntryListener, addSubTableListener, basenameKey, clearFlags, clearPersistent, containsKey, containsSubTable
delete, getBoolean, getBooleanArray, getEntry, getFlags, getHierarchy, getInstance, getKeys
getNumber, getNumberArray, getPath, getRaw, getString, getStringArray, getSubTable, getSubTables
getValue, isPersistent, loadEntries, normalizeKey, putBoolean, putBooleanArray, putNumber, putNumberArray
putRaw, putString, putStringArray, putValue, removeEntryListener, removeTableListener, saveEntries, setDefaultBoolean
setDefaultBooleanArray, setDefaultNumber, setDefaultNumberArray, setDefaultRaw, setDefaultString, setDefaultStringArray, setDefaultValue, setFlags
setPersistent, 

In [46]:
subtables = table.getSubTables()
type(subtables)

list

In [47]:
len(subtables)

279

In [48]:
set(subtables)

{'FMSInfo', 'LiveWindow', 'Ramsete', 'Rev', 'Rev_Climber', 'SmartDashboard'}

In [49]:
s = table.getSubTable('SmartDashboard')

In [50]:
set(s.getSubTables())

{'',
 'AutoFetchBall',
 'AutoPickup',
 'AutoRamsete',
 'AutoRotateIMU',
 'AutoRotateSparkmax',
 'AutoSetPose',
 'AutoShoot',
 'AutonomousStageTwo',
 'AutonomousTwoBall',
 'CompressorToggle',
 'Field',
 'IndexerHold',
 'IntakeMotorToggle',
 'IntakePistonToggle',
 'ShooterToggle',
 'TuneSparkmax',
 'TuneSparkmaxClimber',
 'autonomous routines',
 'path velocity',
 'ramsete path'}

In [51]:
unique = list(set(subtables))
frequency = {}
for item in unique:
    frequency[item] = subtables.count(item)
print("Frequency of items: ", frequency)

Frequency of items:  {'SmartDashboard': 135, 'FMSInfo': 9, 'Ramsete': 9, 'LiveWindow': 92, 'Rev': 17, 'Rev_Climber': 17}


In [52]:
r = table.getSubTable('Ramsete')

#### get all entries - not sure if this is the fastest way to do it

In [53]:
entries = ntinst.getEntries('/', types=0)

In [55]:
sorted_tree = sorted([e.getName() for e in entries])
sorted_tree[::20]

['/FMSInfo/.type',
 '/LiveWindow/Ungrouped/Compressor[0]/.name',
 '/LiveWindow/Ungrouped/Intake/.type',
 '/LiveWindow/Ungrouped/Shooter/.command',
 '/LiveWindow/Ungrouped/Solenoid[0,3]/.type',
 '/LiveWindow/Ungrouped/navX-Sensor[4]/Value',
 '/Rev/pos_arbff',
 '/Rev_Climber/pos_sp',
 '/SmartDashboard/AutoFetchBall/.type',
 '/SmartDashboard/AutoSetPose/.type',
 '/SmartDashboard/Field/.type',
 '/SmartDashboard/TuneSparkmax/.type',
 '/SmartDashboard/climber_velocity',
 '/SmartDashboard/intake_speed']

#### why do i have to take what was a table and get a flat list?

In [56]:
nt_dict = {}
for subtable in sorted(set(subtables)):
    child = [entry[len(subtable)+2:] for entry in sorted_tree if subtable in entry]
    nt_dict.update({subtable:child})
#nt_dict

#### data dump from networktables

In [68]:
d = {}
for ix, item in enumerate(sorted_tree):
    entry = ntinst.getEntry(item)
    entry_val = entry.getValue()
    name = entry.getName()
    # value = val.getDouble() if val.isDouble() else val.getBoolean() if val.isBoolean else val.getString if val.isString else ''
    value = entry_val.value()
   # print(f'Name: {entry.getName():60s} Value:{str(value):20s} \t Age: {int(time.time() - entry_val.time()/1E6):6d}  Type: {entry.getType()}  RPC:{entry_val.isRaw()} Levels:{name[1:].split("/")}')
    d.update({entry.getName(): {'value':str(value), 'age':int(time.time() - entry_val.time()/1E6),  'type': entry.getType(),  'RPC':entry_val.isRaw(), 'levels':name[1:].split("/")}})
df = pd.DataFrame(d).transpose()
pd.set_option('display.max_rows', df.shape[0]+1)
display(df)

,value,age,type,RPC,levels
/FMSInfo/.type,FMSInfo,625,NetworkTableType.kString,False,"[FMSInfo, .type]"
/FMSInfo/EventName,,625,NetworkTableType.kString,False,"[FMSInfo, EventName]"
/FMSInfo/FMSControlData,33.0,590,NetworkTableType.kDouble,False,"[FMSInfo, FMSControlData]"
/FMSInfo/GameSpecificMessage,,625,NetworkTableType.kString,False,"[FMSInfo, GameSpecificMessage]"
/FMSInfo/IsRedAlliance,True,625,NetworkTableType.kBoolean,False,"[FMSInfo, IsRedAlliance]"
/FMSInfo/MatchNumber,0.0,625,NetworkTableType.kDouble,False,"[FMSInfo, MatchNumber]"
/FMSInfo/MatchType,0.0,625,NetworkTableType.kDouble,False,"[FMSInfo, MatchType]"
/FMSInfo/ReplayNumber,0.0,625,NetworkTableType.kDouble,False,"[FMSInfo, ReplayNumber]"
/FMSInfo/StationNumber,1.0,625,NetworkTableType.kDouble,False,"[FMSInfo, StationNumber]"
/LiveWindow/.status/LW Enabled,False,625,NetworkTableType.kBoolean,False,"[LiveWindow, .status, LW Enabled]"


In [19]:
ntdict = {}
for item in sorted_tree:
    entry = ntinst.getEntry(item)
    entry_val = entry.getValue()
    name = entry.getName()
    value = entry_val.value()
    ntdict.update({name:{'nt_path': entry.getName(), 'value':value, 'entry':entry, 'type':entry.getType()}})
    #print(f'Name: {entry.getName():60s} Value:{str(value):20s} \t Age: {int(time.time() - entry_val.time()/1E6):6d}  Type: {entry.getType()}  Levels:{name[1:].split("/")}')
df = pd.DataFrame(ntdict).transpose()
display(df)

,nt_path,value,entry,type
/FMSInfo/.type,/FMSInfo/.type,FMSInfo,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kString
/FMSInfo/EventName,/FMSInfo/EventName,,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kString
/FMSInfo/FMSControlData,/FMSInfo/FMSControlData,32.0,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/FMSInfo/GameSpecificMessage,/FMSInfo/GameSpecificMessage,,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kString
/FMSInfo/IsRedAlliance,/FMSInfo/IsRedAlliance,True,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kBoolean
...,...,...,...,...
/SmartDashboard/ramsete path/default,/SmartDashboard/ramsete path/default,2_circle,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kString
/SmartDashboard/ramsete path/options,/SmartDashboard/ramsete path/options,"[2_circle, 2_meters, PathWeaver, Unnamed.path,...",<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kStringArray
/SmartDashboard/shooter_ready,/SmartDashboard/shooter_ready,False,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kBoolean
/SmartDashboard/shooter_rpm,/SmartDashboard/shooter_rpm,0.0,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble


#### investigate an entry

In [70]:
t = ntinst.getEntry('/SmartDashboard/drive_pose')

In [71]:
t

In [72]:
test_obj = t
attribs = [attrib for attrib in dir(test_obj) if '__' not in attrib]
_ = [print(attrib, end=', ') if (ix+1)%8!=0 else print(attrib, end='\n') for ix, attrib in enumerate(attribs) ]

Flags, addListener, clearFlags, clearPersistent, delete, exists, forceSetBoolean, forceSetBooleanArray
forceSetDouble, forceSetDoubleArray, forceSetRaw, forceSetString, forceSetStringArray, forceSetValue, getBoolean, getBooleanArray
getDouble, getDoubleArray, getFlags, getHandle, getInfo, getInstance, getLastChange, getName
getRaw, getString, getStringArray, getType, getValue, isPersistent, removeListener, setBoolean
setBooleanArray, setDefaultBoolean, setDefaultBooleanArray, setDefaultDouble, setDefaultDoubleArray, setDefaultRaw, setDefaultString, setDefaultStringArray
setDefaultValue, setDouble, setDoubleArray, setFlags, setPersistent, setRaw, setString, setStringArray
setValue, value, 

In [73]:
t.getName()

'/SmartDashboard/drive_pose'

In [74]:
t.getType()

<NetworkTableType.kDoubleArray: 32>

In [75]:
type(t.getType())

_pyntcore._ntcore.NetworkTableType

In [76]:
from _pyntcore._ntcore import NetworkTableType

In [77]:
NetworkTableType.kBoolean

<NetworkTableType.kBoolean: 1>

In [78]:
t.getDoubleArray([1,1,1,])

[6.9991462592082545, 1.1392505223090779, -108.20743560791016]

In [79]:
info = t.getInfo()

In [80]:
[item for item in dir(info) if '_' not in item ]

['entry', 'flags', 'name', 'type']

In [81]:
t.getType()

<NetworkTableType.kDoubleArray: 32>

In [82]:
t.getLastChange()

1648171928695244

In [83]:
val = t.getValue()

In [85]:
test_obj = val
attribs = [attrib for attrib in dir(test_obj) if '__' not in attrib]
_ = [print(attrib, end=', ') if (ix+1)%8!=0 else print(attrib, end='\n') for ix, attrib in enumerate(attribs) ]

getBoolean, getBooleanArray, getDouble, getDoubleArray, getFactoryByType, getRaw, getRpc, getString
getStringArray, isBoolean, isBooleanArray, isDouble, isDoubleArray, isRaw, isRpc, isString
isStringArray, isValid, last_change, makeBoolean, makeBooleanArray, makeDouble, makeDoubleArray, makeRaw
makeRpc, makeString, makeStringArray, makeValue, time, type, value, 

In [86]:
val.getDouble()

1.0502183873084e-311

In [87]:
val.last_change()

1648171928695244

In [88]:
val.time()

1648171928695244

In [89]:
val.time??

Docstring:
time(self: _pyntcore._ntcore.Value) -> int

Get the creation time of the value.

:returns: The time, in the units returned by nt::Now().
Type:      method


In [90]:
val.value()

[6.9991462592082545, 1.1392505223090779, -108.20743560791016]

In [91]:
t2 = ntinst.getEntry('/SmartDashboard/autonomous routines/options')

In [92]:
val = t2.getValue()
val.getStringArray()

['2 ball only', '3 ball lower', '4 ball lower', 'Ramsete Test']

In [93]:
t2.getStringArray([])

['2 ball only', '3 ball lower', '4 ball lower', 'Ramsete Test']

In [94]:
type(val.getStringArray())

list

In [95]:
# generate the dictionary
d={}
levels = [s[1:].split('/') for s in sorted_tree]
for path in levels:
    current_level = d
    for part in path:
        if part not in current_level:
            current_level[part] = {}
        current_level = current_level[part]
#d

In [96]:
levels[100]

['LiveWindow', 'Ungrouped', 'navX-Sensor[4]', 'Value']

In [97]:
d['LiveWindow']['Ungrouped']['navX-Sensor[4]']['Value']

{}

In [98]:
for item in sorted_tree:
    value = ntinst.getEntry(item).getValue().value()
    levels = item[1:].split('/')
    if len(levels) == 2:
        d[levels[0]][levels[1]] = value
    elif len(levels) == 3:
        d[levels[0]][levels[1]][levels[2]] = value
    elif len(levels) == 4:
        d[levels[0]][levels[1]][levels[2]][levels[3]] = value

In [99]:
d

{'FMSInfo': {'.type': 'FMSInfo',
  'EventName': '',
  'FMSControlData': 33.0,
  'GameSpecificMessage': '',
  'IsRedAlliance': True,
  'MatchNumber': 0.0,
  'MatchType': 0.0,
  'ReplayNumber': 0.0,
  'StationNumber': 1.0},
 'LiveWindow': {'.status': {'LW Enabled': False},
  'Ungrouped': {'.type': 'LW Subsystem',
   'AnalogInput[0]': {'.name': 'AnalogInput[0]',
    '.type': 'Analog Input',
    'Value': 0.0},
   'Climber': {'.command': 'none',
    '.default': 'none',
    '.hasCommand': False,
    '.hasDefault': False,
    '.name': 'Climber',
    '.type': 'Subsystem'},
   'Compressor[0]': {'.name': 'Compressor[0]',
    '.type': 'Compressor',
    'Enabled': False,
    'Pressure switch': False},
   'DifferentialDrive[1]': {'.actuator': True,
    '.name': 'DifferentialDrive[1]',
    '.type': 'DifferentialDrive',
    'Left Motor Speed': 0.0,
    'Right Motor Speed': 0.0},
   'Indexer': {'.command': 'none',
    '.default': 'none',
    '.hasCommand': False,
    '.hasDefault': False,
    '.name':

In [100]:
def depth(d):
    if isinstance(d, dict):
        return 1 + (max(map(depth, d.values())) if d else 0)
    return 0

In [101]:
depth(d)

6

In [102]:
widget_dict = {'qlabel_camera_view': {'widget':'self.qlabel_camera_view', 'table':''},
        'qlabel_climber_indicator':{'widget':'self.qlabel_climber_indicator', 'table':''},
        }

In [103]:
for key, value in widget_dict.items():
    print(key, value)

qlabel_camera_view {'widget': 'self.qlabel_camera_view', 'table': ''}
qlabel_climber_indicator {'widget': 'self.qlabel_climber_indicator', 'table': ''}
